In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential 
import pandas as pd
from tensorflow.keras.utils import plot_model
import numpy as np
import pandasql as ps
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
from scipy.stats import chi2_contingency
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split 
import keras_tuner as kt

In [2]:
data= pd.read_csv(r'C:\Users\liatw\OneDrive\Documents\clinical data project\heart_failure_clinical_records_dataset.csv')
data

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


In [3]:
###########################################################
### Building predictive model using Logistic regression ###
###########################################################

# Build ordinal variables from continous variables by dividing continous variables into quartiles
num_var_list = ['age','creatinine_phosphokinase','ejection_fraction','platelets','serum_creatinine',
                'serum_sodium','time'] 
for var in num_var_list:
    data[var+'_'+'q'] = pd.qcut(data[var],4,labels=False)
data

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT,age_q,creatinine_phosphokinase_q,ejection_fraction_q,platelets_q,serum_creatinine_q,serum_sodium_q,time_q
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1,3,2,0,2,3,0,0
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1,1,3,1,2,1,1,0
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1,2,1,0,0,2,0,0
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1,0,0,0,0,3,1,0
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1,2,1,0,3,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0,2,0,1,0,1,3,3
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0,1,3,1,2,2,2,3
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0,0,3,3,3,0,2,3
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0,0,3,1,0,2,2,3


In [4]:
## univariate analysis 
# The univariate analysis includes two outputs:
# Univariate table - in which the percentage of death is calculated for each of the variables categories.  
# pvalues_table - contains the P-values of each variable for predicting death.     

var_list = ['age_q','creatinine_phosphokinase_q','ejection_fraction_q','serum_creatinine_q','serum_sodium_q','time_q',
                  'anaemia','diabetes','high_blood_pressure','sex','smoking']  

univariate_table = pd.DataFrame(columns = ['var_name', 'var_cat', 'var_mean']) 
pvalues = []
for var in var_list:
    q_univariate = f""" select {var} as var_cat,
                              avg(death_event) as var_mean
                       from data
                       group by {var}"""

    univariate_table_ = pysqldf(q_univariate) 
    univariate_table_['var_name'] = [var]*univariate_table_.shape[0]
    univariate_table = pd.concat([univariate_table, univariate_table_])
    
    stat, p, dof, expected = chi2_contingency(pd.crosstab(index=data[var], columns=data['DEATH_EVENT']))
    pvalues.append(p)
        
univariate_table
pvalues_table = pd.DataFrame (pvalues, columns = ['pvalue'])
pvalues_table['var_name'] = var_list
print(pvalues_table)

          pvalue                    var_name
0   6.027344e-05                       age_q
1   1.997368e-01  creatinine_phosphokinase_q
2   2.710166e-07         ejection_fraction_q
3   2.515347e-10          serum_creatinine_q
4   4.141172e-04              serum_sodium_q
5   4.846835e-27                      time_q
6   3.073161e-01                     anaemia
7   1.000000e+00                    diabetes
8   2.141034e-01         high_blood_pressure
9   1.000000e+00                         sex
10  9.317653e-01                     smoking


In [5]:
# Preparing the X and y inputs for the model:
# X- a table of the explanatory variables which were found significant in the univariate analysis. 
# y - an output vector 
X = data.loc[:, data.columns != 'DEATH_EVENT']
y = data.loc[:, data.columns == 'DEATH_EVENT']

X=X[['age_q','ejection_fraction_q','serum_creatinine_q','serum_sodium_q','time_q']]
y = y.values.ravel() 

In [6]:
# Spliting X and y into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Applying Logistic regression on train data
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression()

In [7]:
# Calculating prediction accuracy on test data 
y_pred = logreg.predict(X_test)
metrics.accuracy_score(y_test, y_pred, normalize=True)

0.8333333333333334

In [9]:
###########################################################
### Building predictive model using Neural networks ###
###########################################################
## Initial model 
# Setting an architecture of the network
model = Sequential()
model.add(keras.layers.Dense(12, input_shape=(5,), activation='relu'))
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [10]:
# Compiling the model
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])

In [11]:
# Training the model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
7/7 [==============================] - 2s 87ms/step - loss: 0.6074 - accuracy: 0.6794 - val_loss: 0.6181 - val_accuracy: 0.6556
Epoch 2/20
7/7 [==============================] - 0s 12ms/step - loss: 0.5606 - accuracy: 0.7225 - val_loss: 0.5887 - val_accuracy: 0.6667
Epoch 3/20
7/7 [==============================] - 0s 11ms/step - loss: 0.5313 - accuracy: 0.7512 - val_loss: 0.5712 - val_accuracy: 0.6778
Epoch 4/20
7/7 [==============================] - 0s 12ms/step - loss: 0.5108 - accuracy: 0.7703 - val_loss: 0.5620 - val_accuracy: 0.6778
Epoch 5/20
7/7 [==============================] - 0s 11ms/step - loss: 0.4975 - accuracy: 0.7703 - val_loss: 0.5569 - val_accuracy: 0.6889
Epoch 6/20
7/7 [==============================] - 0s 11ms/step - loss: 0.4871 - accuracy: 0.7895 - val_loss: 0.5521 - val_accuracy: 0.7000
Epoch 7/20
7/7 [==============================] - 0s 12ms/step - loss: 0.4778 - accuracy: 0.7895 - val_loss: 0.5445 - val_accuracy: 0.7000
Epoch 8/20
7/7 [===========

In [12]:
## Improving the initial model
# The accuracy of the initial model was much lower than the logistic regression model's accuracy (71.1% vs. 83.3%). 
# Therefore, I was trying to improve the model by hyperparameter search.
# The hyperparameters that were searched for were the number of layers and the learning rate.

def model_builder(hp):
  model = Sequential()
  model.add(keras.layers.Dense(12, input_shape=(5,), activation='relu'))

  # Tune the number of dense layers
  for i in range(hp.Int('num_layers', 1, 3)):
    
    # Tune the number of units in the each dense layer
    hp_units = hp.Int('units_'+str(i), min_value=1, max_value=50, step=10)
    model.add(tf.keras.layers.Dense(units=hp_units, activation='relu'))
    
    # Tune the dropout rate in the each dense layer
        #hp_dropout = hp.Float('rate', min_value=0.0, max_value=0.5, step=0.1)
        #model.add(tf.keras.layers.Dropout(hp_dropout))
    
  # Add dense output layer
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

  # Tune the learning rate for the optimizer
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4])
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='binary_crossentropy',
                metrics=['accuracy'])

  return model

In [51]:
tuner = kt.RandomSearch(model_builder,
  objective='val_accuracy',
  max_trials=10,
  executions_per_trial=3,
  directory='keras_tuner',
  project_name='clinical_data_project',
  overwrite=True)

In [52]:
# Spliting X and y into train, validation and test:
# At the first step, X and y are splitted into train and test
# At the second step, train data is splitted into train and validation for the hyperparameters search process
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
tuner.search(X_train, y_train, epochs=20, validation_split=0.2)

Trial 10 Complete [00h 00m 06s]
val_accuracy: 0.8641975522041321

Best val_accuracy So Far: 0.8703703880310059
Total elapsed time: 00h 00m 53s
INFO:tensorflow:Oracle triggered exit


In [53]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

In [54]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2)

Epoch 1/20
7/7 [==============================] - 1s 34ms/step - loss: 0.6595 - accuracy: 0.7442 - val_loss: 0.4692 - val_accuracy: 0.8148
Epoch 2/20
7/7 [==============================] - 0s 3ms/step - loss: 0.4275 - accuracy: 0.8233 - val_loss: 0.5458 - val_accuracy: 0.8148
Epoch 3/20
7/7 [==============================] - 0s 5ms/step - loss: 0.3993 - accuracy: 0.8465 - val_loss: 0.5481 - val_accuracy: 0.7407
Epoch 4/20
7/7 [==============================] - 0s 4ms/step - loss: 0.3734 - accuracy: 0.8000 - val_loss: 0.4779 - val_accuracy: 0.7963
Epoch 5/20
7/7 [==============================] - 0s 5ms/step - loss: 0.3118 - accuracy: 0.8698 - val_loss: 0.5513 - val_accuracy: 0.8333
Epoch 6/20
7/7 [==============================] - 0s 4ms/step - loss: 0.3154 - accuracy: 0.8651 - val_loss: 0.5364 - val_accuracy: 0.8333
Epoch 7/20
7/7 [==============================] - 0s 5ms/step - loss: 0.2909 - accuracy: 0.8651 - val_loss: 0.5631 - val_accuracy: 0.8704
Epoch 8/20
7/7 [=================

In [55]:
# Calculate prediction accuracy on the test data
model.evaluate(X_test, y_test, return_dict=True)

1/1 [==============================] - 0s 16ms/step - loss: 0.4325 - accuracy: 0.8667


{'loss': 0.4324561655521393, 'accuracy': 0.8666666746139526}